In [4]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [80]:
ratings = pd.read_csv(path/'u.data',
                     delimiter='\t',
                     header=None,
                     names=['user','movie','rating','timestamp'])
# 기본 테이블 path/'u.data'

ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [81]:
# SF, 액션, 고전
user1 = np.array([0.98, 0.9, -0.9])
user2 = np.array([0.9, 0.8, -0.6])

# user1과 user2의 유사성은 접곰으로 구한다.
(user1 * user2).sum()

2.1420000000000003

In [82]:
movies = pd.read_csv(path/'u.item',
                     delimiter='|',
                     encoding='latin-1',
                     usecols=(0,1),
                     names=['movie','title'],
                     header=None)
# 영화 ID에 해당하는 제목 경로 path/'u.item'

movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [83]:
# 이 제목 정보를 ratings 테이블과 결합

ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [84]:
# CallabDataLoaders는
# 디폴트로 1번열을 사용자, 2번열을 항목(영화), 3번열을 점수로 사용한다.
# 2번열을 항목 대신 제목으로 바꾸려 한다.

dls = CollabDataLoaders.from_df(ratings,
                                item_name='title',
                                bs=64)
dls.show_batch()

# 열 변경은 순서대로 user_name, item_name, rating_name 매개변수를 바꾼다.

,user,title,rating
0,880,Dave (1993),4
1,788,Paris Is Burning (1990),3
2,833,Boogie Nights (1997),5
3,169,Casablanca (1942),3
4,661,Enchanted April (1991),5
5,464,Gattaca (1997),4
6,870,"Paper, The (1994)",2
7,843,Heavy Metal (1981),3
8,871,Top Gun (1986),5
9,234,Four Weddings and a Funeral (1994),3


In [92]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_fatcors = 5
print('n_users:', n_users)

torch.randn(n_users, n_fatcors) # 임의로 n_users*5 행렬 만든다.

n_users: 944


tensor([[ 0.6470, -0.2019, -0.6334, -2.2870,  0.6082],
        [ 0.2917, -0.0224, -0.7459, -0.0506, -0.8882],
        [ 1.4981,  0.1669,  1.0496,  0.2169,  0.2054],
        ...,
        [ 0.5434,  0.0423,  1.6387, -0.9341,  0.6146],
        [ 0.1950,  0.7329, -0.5764, -0.3876,  3.2328],
        [ 0.6268, -2.1163,  0.1140, -0.5319, -0.6411]])

In [108]:

class DotProduct(Module):
    # 임베딩 행렬 초기화
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
    
    # 임베딩 행렬 완성, 점곱
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users*movies).sum(dim=1)

## 1. 모델에 넘겨줄 데이터 dls

- **X: 예측에 필요한 데이터**
    - 사용자ID.
    - 영화ID.

- **Y: 타겟**
    - 사용자ID별 평점 점수


## 2. 모델이 할 일
- **임베딩**
    - 사용자ID를 수치(특성)로 나타낸다.
    - 영화ID를 수치(특성)로 나타낸다.

- **점곱**
    - 두 수치의 점곱을 구하여 합산한다.
    - 이것는 사용자ID별 평점 예측 점수가 된다.

- **손실 줄이기**
    - 사용자ID별 평점 예측 점수와 실제 평점 점수를 비교하여 손실을 줄여간다.

## 3. 임베딩
- **범주형(ID등)** 을 **수치형**으로 바꾼다.
- 예를 들어, 사용자수를 100, 잠재요소개수를 5라고 하자.

1. 임베딩행렬 user_fatcors 초기화
    - (100*5) 모양이야.
<br><br>
2. 사용자ID 넘기기
    - dls의 0번째 열 x[:, 0]은 [2, 10, 7 ..]와 같은 사용자ID야.
    - 이 ID가 행렬의 인덱스가 돼.
    - 두 번째 사용자ID는 행렬의 두 번째 벡터가 돼.
    - [0.64, -0.2, -0.6, 0.5, 0.1]
<br><br>
3. 임베딩행렬 user_fatcors 완성
    - 사용자ID와 임베딩행렬이 매칭되면 행렬이 완성돼.
    - (100*5) 모양이야.
<br><br>
4. 임베딩 행렬 movie_factors 완성
    - (100*5) 모양이야.
<br><br>
5. 점곱
    - user_factors와 movie_factors를 곱한 후 요소를 합산해.
    - 이 값이 각 사용자의 영화 평점 예측값이 돼.
    - (100) 모양이야.
    
    
## 4. 점곱
user_fatcors(100*5) 와 user_factors(100*5)를 어떻게 곱할 수 있지?
<br>
NumPy의 `np.dot()` 함수와 벡터의 점곱(dot product)은 다르다.

1. **NumPy의 `np.dot()` 함수**:
   - 일반적인 행렬 곱셈. 첫 번째 행렬의 열과 두 번째 행렬의 행이 곱해진다.

2. **벡터의 점곱**:
   - **동일한 위치**에 있는 요소들을 곱하여 구한다.
   - 두 벡터 `[a, b, c]`와 `[d, e, f]`의 점곱은 `a*d + b*e + c*f`이다.

In [110]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [111]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.322955,1.290159,00:14
1,1.097970,1.084686,00:12
2,0.950265,0.977421,00:12
3,0.826734,0.890247,00:13
4,0.763387,0.876253,00:13
